## Music Evaluation

In [1]:
import json
from argparse import ArgumentParser
import midi
import glob
import copy
import os
import numpy as np
import pretty_midi
from pprint import pprint
import pickle
from mgeval.mgeval import core, utils
from sklearn.model_selection import LeaveOneOut

In [2]:
# midi file이 들어있는 dict path를 넣어주세요!

set1dir = '/home/bang/PycharmProjects/MusicTransformer-Pytorch/statistic_file/MT_classic_pop_static/classic'
set2dir = '/home/bang/PycharmProjects/MusicTransformer-Pytorch/statistic_file/MT_classic_pop_static/pop'

In [3]:
set1 = glob.glob(os.path.join(set1dir, '*'))
set2 = glob.glob(os.path.join(set2dir, '*'))

## Number of samples

- set1: classic midi files
- set2: pop midi files

In [4]:
print("set1 size :", len(set1), ", set2 size :", len(set2))

set1 size : 1281 , set2 size : 829


In [5]:
num_samples = min(len(set2), len(set1))

## metrics

- total used pitch
- pitch_range
- avg pitch shift
- avg IOI
- total_pitch class histogram

In [6]:
evalset = { 
            'total_used_pitch': np.zeros((num_samples, 1))
          , 'pitch_range': np.zeros((num_samples, 1))
          , 'avg_pitch_shift': np.zeros((num_samples, 1))
          , 'avg_IOI': np.zeros((num_samples, 1))
          #, 'total_used_note': np.zeros((num_samples, 1))
          # , 'bar_used_pitch': np.zeros((num_samples, args.num_bar, 1))
          # , 'bar_used_note': np.zeros((num_samples, args.num_bar, 1))
          , 'total_pitch_class_histogram': np.zeros((num_samples, 12))
          # , 'bar_pitch_class_histogram': np.zeros((num_samples, args.num_bar, 12))
          #, 'note_length_hist': np.zeros((num_samples, 12))
          #, 'pitch_class_transition_matrix': np.zeros((num_samples, 12, 12))
          #, 'note_length_transition_matrix': np.zeros((num_samples, 12, 12))
          }

In [7]:
metrics_list = list(evalset.keys())

single_arg_metrics = (
    [ 'total_used_pitch'
    , 'avg_IOI'
    , 'total_pitch_class_histogram'
    , 'pitch_range'
    ])

set1_eval = copy.deepcopy(evalset)
set2_eval = copy.deepcopy(evalset)

sets = [ (set1, set1_eval), (set2, set2_eval) ]

## 평가 진행

In [ ]:
# Extract Fetures
for _set, _set_eval in sets:
    for i in range(0, num_samples):
        feature = core.extract_feature(_set[i])
        for metric in metrics_list:
            evaluator = getattr(core.metrics(), metric)
            if metric in single_arg_metrics:
                tmp = evaluator(feature)
            # elif metric in bar_metrics:
            #     # print(metric)
            #     tmp = evaluator(feature, 0, args.num_bar)
            # print(tmp.shape)
            else:
                tmp = evaluator(feature, 1)
            _set_eval[metric][i] = tmp

## 평가 결과

In [ ]:
set1_TUP = (set1_eval['total_used_pitch']).sum(-1).mean()
set2_TUP = (set2_eval['total_used_pitch']).sum(-1).mean()

print("set1 total used pitch :", set1_TUP)
print("set2 total used pitch :", set2_TUP)

In [ ]:
set1_TCU = (set1_eval['total_pitch_class_histogram']!=0).sum(-1).mean()
set2_TCU = (set2_eval['total_pitch_class_histogram']!=0).sum(-1).mean()

print("set1_TCU :", set1_TCU)
print("set2_TCU :", set2_TCU)

In [ ]:
set1_PR = (set1_eval['pitch_range']).mean()
set2_PR = (set2_eval['pitch_range']).mean()

print("set1_PR :", set1_PR)
print("set2_PR :", set2_PR)

In [ ]:
set1_APS = set1_eval['avg_pitch_shift'].sum(-1).mean()
set2_APS = set2_eval['avg_pitch_shift'].sum(-1).mean()

print("set1_APS :", set1_APS)
print("set2_APS :", set2_APS)

In [ ]:
set1_IOI = set1_eval['avg_IOI'].sum(-1).mean()
set2_IOI = set2_eval['avg_IOI'].sum(-1).mean()

print("set1_IOI :", set1_IOI)
print("set2_IOI :", set2_IOI)